In [1]:
try:
  # %tensorflow_version only existis in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf 
print(tf.__version__)

from tensorflow import keras
from tensorflow.keras import layers
#import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras import Model

2.5.0


# Transfer Learning

In [2]:
vgg_model = tf.keras.applications.vgg16.VGG16()
print(type(vgg_model))
vgg_model.summary()

<class 'tensorflow.python.keras.engine.functional.Functional'>
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
________________________________________________________________

In [3]:
model = keras.models.Sequential()
for layer in vgg_model.layers[0:-1]:
    model.add(layer)

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [5]:
for layer in model.layers:
    layer.trainable = False
model.summary()

#Setting the network to not trainable so, training of would not be necessary for this part

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [6]:
model.add(layers.Dense(2))
#adding last layer to ensure two outputs

In [7]:
# loss and optimizer
loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
optim = keras.optimizers.Adam(learning_rate = 0.001)
metrics = [keras.metrics.BinaryAccuracy()]

model.compile(optimizer= optim, loss = loss, metrics = metrics)

In [8]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [9]:
#Generate batches of tensor image data with real-time data augmentation
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function = preprocess_input)

In [10]:
names = ['Flooding', 'Regular_State']

In [15]:
train_batches = train_gen.flow_from_directory(
    ('C:/Users/Nurcan Düzenli/MLME_ExtractionClassification/train'),
    target_size=(1664, 1024),
    color_mode="grayscale",
    batch_size=32,
    class_mode="binary",
    shuffle=True,
    seed=42,
    classes = names
)
#change of directory is required

Found 3610 images belonging to 2 classes.


In [16]:
val_batches = valid_gen.flow_from_directory(
    ('C:/Users/Nurcan Düzenli/MLME_ExtractionClassification/validation'),
    target_size=(1664, 1024),
    color_mode="grayscale",
    batch_size=32,
    class_mode="binary",
    shuffle=False, #No shuffling
    seed=42,
    classes = names
)
#change of directory is required

Found 774 images belonging to 2 classes.


In [17]:
test_batches = test_gen.flow_from_directory(
    ('C:/Users/Nurcan Düzenli/MLME_ExtractionClassification/test'),
    target_size=(1664, 1024),
    color_mode="grayscale",
    batch_size=32,
    class_mode="binary",
    shuffle=False, #No shuffling
    seed=42,
    classes = names
)
#change of directory is required

Found 774 images belonging to 2 classes.


In [19]:
epochs = 3
model.fit(train_batches, epochs = epochs, validation_data = val_batches)

IndexError: index 1 is out of bounds for axis 2 with size 1